In [ ]:
import pandas as pd
#provaxxers
#provaxxersTweets = pd.read_csv('./datasets/provaxxers/provaxxersTweets.csv', low_memory=False)

#antivaxxers
antivaxxersTweets = pd.read_csv('./datasets/antivaxxers/antivaxxersTweets.csv', low_memory=False)

#antisinovaxxers
#antisinovaxxersTweets = pd.read_csv('./datasets/antisinovaxxers/antisinovaxxersTweets.csv', low_memory=False)

In [ ]:
rawTweets = antivaxxersTweets

In [ ]:
#start_date ='2020-02-29T00:00:00.000Z'
#end_date = '2021-05-04T00:00:00.000Z'

start_date ='2021-03-00T00:00:00.000Z'
end_date = '2021-05-04T00:00:00.000Z'

mask = (rawTweets['created_at'] > start_date) & (rawTweets['created_at'] <= end_date)
filteredTweets = rawTweets.loc[mask]

filteredTweets = filteredTweets.sort_values(['created_at']).reset_index().drop(columns=["index"])

In [ ]:
filteredTweets

In [ ]:
import botometer
rapid_api_key = ''
twitter_app_auth = {
    'consumer_key': '',
    'consumer_secret': '',
    'access_token': '',
    'access_token_secret': '',
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapid_api_key,
                          **twitter_app_auth)


In [ ]:
result = bom.check_account(1255307463034843136)

In [ ]:
result

In [ ]:
result

In [ ]:
import nltk
from nltk import tokenize

import numpy as np 
from string import punctuation
import unidecode
stemmer = nltk.RSLPStemmer()

# Removendo hashtags, menções a usuários, numeros, termos curtos e links

def proccess_text(raw_text):
    
    twitterData = pd.DataFrame(raw_text) 
    twitterData['processed_text'] = twitterData.text.str.replace(r'(http\S+)', '') \
                                                    .str.replace(r'@[\w]*', '') \
                                                    .str.replace(r'#[\w]*','') 

    textWords = ' '.join([text for text in twitterData.processed_text])

    # Removendo acentuação
    textWords = [unidecode.unidecode(text) for text in twitterData.processed_text ]

    # Criando lista com palavras e caracteres (stopwords) a serem removidos do texto
    stopWords = nltk.corpus.stopwords.words("portuguese")

    # Separando a pontuação das palavras
    punctSeparator = tokenize.WordPunctTokenizer()
    punctuationList = list()
    for punct in punctuation:
        punctuationList.append(punct)

    #personalList=['predictions','tomar cu']    

    #stopWords =  stopWords + punctuationList + personalList
    stopWords =   punctuationList + stopWords


    # Iterando o texto removendo as stopwords
    trasnformedText = list()
    
    for text in textWords:
        newText = list()   
        text = text.lower()
        textWords = punctSeparator.tokenize(text)
        for words in textWords:
             if words not in stopWords:
                #newText.append(stemmer.stem(words))
                newText.append(words)
        trasnformedText.append(' '.join(newText))
    twitterData.processed_text = trasnformedText

   
    twitterData.processed_text = twitterData.processed_text.str.replace(r"[^a-zA-Z#]", " ") \
                                                           .replace(r"k\k", " ") #\
                                                           #.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) 
    
    return twitterData.processed_text


tweets = {'created_at': filteredTweets.created_at, 'text': filteredTweets.text,'author_id':filteredTweets.author_id}
rawTweets = pd.DataFrame(tweets)

rawTweets['processed_text'] = proccess_text(rawTweets.text)

processedTweets = rawTweets.drop(columns=["text"])

trasnformedText = list()
for phrase in processedTweets.processed_text:
    newPhrase = list()   
    newPhrase.append(' '.join(phrase.split()))
    for words in newPhrase:
        trasnformedText.append(''.join(newPhrase))
processedTweets.processed_text = trasnformedText

index=[x for x in processedTweets.index if processedTweets.processed_text[x].count(' ') < 3]
processedTweets = processedTweets.drop(index)

removeEmpty  = processedTweets.processed_text != ' '
processedTweets = processedTweets[removeEmpty]

processedTweets.reset_index(inplace=True)

tweets = {'created_at': processedTweets.created_at, 'text': processedTweets.processed_text, 'author_id':processedTweets.author_id}
docs = pd.DataFrame(tweets)

docs = docs.sort_values(['created_at']).reset_index()

docs = docs.drop(columns=["index"])

docs

# BERTopic

<b>Custom sub-models</b>
<ul>
    <li>UMAP: Dimensionality reduction</li>
    <li>HDBSCAN: Documents clustering</li>
    <li>CountVectorizer: Extract all possible words</li>
</ul> 

In [ ]:
import torch
if torch.cuda.is_available():          
    device = torch.device("cuda")    
    print('There are %d GPU(s) available.' % torch.cuda.device_count())    
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
#from sentence_transformers import SentenceTransformer, util
#bert_model = SentenceTransformer("xlm-r-bert-base-nli-stsb-mean-tokens", device="cuda")

In [ ]:
from umap import UMAP
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine')

In [ ]:
#import hdbscan

#hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, 
#                                metric='euclidean',                      
#                                cluster_selection_method='eom', prediction_data=True)

In [ ]:
#from sklearn.feature_extraction.text import CountVectorizer

#vectorizer_model = CountVectorizer(ngram_range=(1, 2),min_df=5)

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(#embedding_model=bert_model,
                       language = "multilingual",
                       top_n_words=10,
                       n_gram_range=(1, 2),
                       #min_topic_size=65,   
                       #nr_topics = "auto",
                       #umap_model=umap_model,
                       #hdbscan_model=hdbscan_model,
                       #vectorizer_model=vectorizer_model,
                       low_memory=True,
                       calculate_probabilities=False, 
                       verbose=True)#.fit(processedTweets.text)

In [ ]:
topics, probs = topic_model.fit_transform(docs.text)

In [ ]:
#from bertopic import BERTopic
# Save model
#topic_model.save("./models/antivaxxers_model")

# Load model
#topic_model = BERTopic.load("./models/provaxxers_model")
#topics = topic_model.get_topics()

In [ ]:
freq = topic_model.get_topic_info(); freq.head(11)

In [ ]:
#topic_model.get_topic(9)

In [ ]:
topic_model.visualize_topics()

In [ ]:
#topic_model.visualize_hierarchy(top_n_topics=30)

In [ ]:
#similar_topics, similarity = topic_model.find_topics("doria", top_n=30); similar_topics

In [ ]:
topic_docs = {topic: [] for topic in set(topics)}
for topic, doc in zip(topics, docs.text):
    topic_docs[topic].append(doc)
    
    
topic_docs[0]

## Topic Reduction

<ul>
    <li>Manual Topic Reduction</li>
    <li>Automatic Topic Reduction</li>
    <li><b>Topic Reduction after Training</b></li>
</ul> 


In [ ]:
newTopics, newProbs = topic_model.reduce_topics(docs.text, topics, probs, nr_topics=3) 

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_docs = {topic: [] for topic in set(newTopics)}
for topic, doc in zip(newTopics, docs.text):
    topic_docs[topic].append(doc)

In [ ]:
len(topic_docs[8])

In [ ]:
topic_docs[8]

In [ ]:
#topic_model.visualize_hierarchy(top_n_topics=30)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(30)

In [ ]:
 topic_model.get_topic(3)

# Dynamic Topic Modeling

[Dou et al. 2012] define um evento como ”uma ocorrencia que causa mudancas no volume de dados de texto que discutem o topico associado em um momento especifico. Logo, eventos podem ser vistos como um resumo sucinto dos fluxos de informacoes nas mıdias sociais, revelando a evolucao de fenomenos sociais ao longo de um determinado periodo de tempo.

In [ ]:
timestamps = docs.created_at.to_list()
tweets = docs.text.to_list()

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                topics=newTopics, 
                                                #topics=topics,
                                                timestamps=timestamps, 
                                                global_tuning=True,
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=30)